In [0]:
from pathlib import Path
import pandas as pd
import csv
import os
from google.colab import drive
from fastai.vision import *

In [0]:
s = b'\x80\x04\x95c\x03\x00\x00\x00\x00\x00\x00\x8c\rpydrive.drive\x94\x8c\x0bGoogleDrive\x94\x93\x94)\x81\x94}\x94(\x8c\x04attr\x94}\x94\x8c\x05dirty\x94}\x94\x8c\x04http\x94N\x8c\x04auth\x94\x8c\x0cpydrive.auth\x94\x8c\nGoogleAuth\x94\x93\x94)\x81\x94}\x94(\x8c\x0chttp_timeout\x94Nh\x05}\x94(h\tN\x8c\rclient_config\x94}\x94\x8c\x08settings\x94}\x94(\x8c\x15client_config_backend\x94\x8c\x04file\x94\x8c\x12client_config_file\x94\x8c\x13client_secrets.json\x94\x8c\x10save_credentials\x94\x89\x8c\x0boauth_scope\x94]\x94\x8c%https://www.googleapis.com/auth/drive\x94au\x8c\x0bcredentials\x94\x8c\x13oauth2client.client\x94\x8c\x11GoogleCredentials\x94\x93\x94)\x81\x94}\x94(\x8c\x0caccess_token\x94N\x8c\tclient_id\x94\x8c&32555940559.apps.googleusercontent.com\x94\x8c\rclient_secret\x94\x8c\x18ZmssLNjJy2998hD4CTg2ejr2\x94\x8c\rrefresh_token\x94\x8cg1//04JAr-Rd85CC1CgYIARAAGAQSNwF-L9IrA5BeS07fWfeLWvn7YCrJVGnWDAX_uKSLUdwpN7u-RkG22wAA4PDI_55Kt-jokJbnUT0\x94\x8c\x0ctoken_expiry\x94N\x8c\ttoken_uri\x94\x8c#https://oauth2.googleapis.com/token\x94\x8c\nuser_agent\x94\x8c\x15Python client library\x94\x8c\nrevoke_uri\x94\x8c$https://oauth2.googleapis.com/revoke\x94\x8c\x08id_token\x94N\x8c\x0cid_token_jwt\x94N\x8c\x0etoken_response\x94N\x8c\x06scopes\x94\x8f\x94\x8c\x0etoken_info_uri\x94N\x8c\x07invalid\x94\x89ubuh\x07}\x94ubub.'
drive = pickle.loads(s)

In [0]:
classes = ['photo_ID', 'earthquake', 'flooding', 'fire', 'hurricane', 'bridge', 'normal', 'building_damage', 'roads', 'low_vegetation',
           'high_vegetation', 'river', 'rough_terrain']

tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [0]:
os.mkdir("Sys_Lab_2")
os.chdir("Sys_Lab_2")
os.mkdir("first_100s")

In [0]:
file6 = drive.CreateFile({'id': '1--FUzxvVhJ3bvuYQIH9ho3VC--lcsXrQ'})
file6.GetContentFile('multi_annotations.csv')

In [0]:
os.chdir("first_100s")

In [0]:
fileList = drive.ListFile({'q': "'1GWv0hVoqEo-3a1px58xOgVBO-4ezRK9r' in parents and trashed=false"}).GetList() # that weird jjumble of stuff is the fodler id it is in the folder url
for file in fileList:
    file6 = drive.CreateFile({'id': file['id']})
    file6.GetContentFile(file['title'])

In [0]:
os.chdir("..")
os.chdir("..")
path_to_dataset = Path("Sys_Lab_2")

In [0]:
np.random.seed(42) # set random seed so we always get the same validation set
src = (ImageList.from_csv(path_to_dataset, 'multi_annotations.csv', folder='first_100s')
       # Load data from df
       .split_by_rand_pct(0.2)
       # split data into training and validation set (20% validation)
       .label_from_df(label_delim=' ')
       # label data using the tags column (second column is default)
)

In [0]:
data = (src.transform(tfms, size=128)
        .databunch(bs=64).normalize(imagenet_stats))

# create metrics
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
# create cnn with the resnet50 architecture
learn = cnn_learner(data, models.resnet50, metrics=[acc_02, f_score])

lr = 0.01 # chosen learning rate

In [0]:
learn.load('last')

In [0]:
learn.fit_one_cycle(1, lr) # train model for 4 epochs
learn.save('last')

In [0]:
lastScore = learn.validate(learn.data.valid_dl)[1].item()
learn.load('best_model/export')
bestScore = learn.validate(learn.data.valid_dl)[1].item()
learn.load('last')

In [0]:
if lastScore > bestScore:
    learn.save('best_model/best')
    learn.export('best_model/export.pkl')
